In [27]:
import pickle
import random
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta, FR
from typing import Tuple

from itertools import combinations
from collections import OrderedDict, defaultdict

In [57]:
people = ["Bridget", "Bud", "Carly", "Cody", "Denis", "Eunice", "Jie", "Jonathan", "Kelly", "Kirtiraj", "Kyle B.", "Kyle C.", "Mohar", "Piyush", "Stan"]
teams = {"Team 1": ["Denis", "Mohar", "Jie", "Cody"]}

history = {
    date(2023, 1, 16) : frozenset({frozenset({"Mohar", "Kirtiraj", "Jie"}), 
                      frozenset({"Kelly", "Kyle C.", "Jonathan"}),
                      frozenset({"Eunice", "Denis", "Carly"}),
                      frozenset({"Bridget", "Kyle B.", "Piyush"}),
                      frozenset({"Cody", "Stan", "Bud"})}),
}

today = date.today()
end_period = today 

In [38]:
# date.today()
yesterday = date.today() - timedelta(1)
print(yesterday)
print(date.today() < yesterday)

2023-01-27
False


In [48]:
today

datetime.date(2023, 1, 28)

In [49]:
date(2022, 12, 25)


datetime.date(2022, 12, 25)

In [41]:
type(today)

datetime.date

In [39]:
today = date.today()
# get the next next Friday
end_date = today + relativedelta(weekday=FR(+2))
today_mdy = today.strftime("%m/%d/%Y")
end_mdy = end_date.strftime("%m/%d/%Y")
print(today_mdy, end_mdy)

01/28/2023 02/10/2023


In [52]:
for team_name, team in teams.items():
    for pair in combinations(team, 2):
        print(pair)

('Denis', 'Mohar')
('Denis', 'Jie')
('Denis', 'Cody')
('Mohar', 'Jie')
('Mohar', 'Cody')
('Jie', 'Cody')


In [54]:
pairs_so_far = get_pairs_so_far(history, teams)

In [53]:
def get_pairs_so_far(history: dict, teams: dict) -> defaultdict:
    """
    Description:
        Based on history, get # of occurences of people meeting

    Args:
        history (dict of datetime.date : frozenset(frozenset)): 
            the history of past groups

        teams (dict of str: list[str]): 
            dictionary outlining prexisting teams. We include this to ensure people 
            initially are not matched with people they already see on a regular cadence


    Returns:
        pairs_so_far (defaultdict of frozenset: int):
            Default dictionary where keys are frozenset of a pair and value is 
            # of past interactions, default value of 0
    """
    
    pairs_so_far = defaultdict(int)
    
    # count occurences in history
    for time, val in history.items():
        for elem in list(val):
            for pair in combinations(elem, 2):
                pairs_so_far[frozenset(pair)] += 1
    
    # count occurences in teams
    for team_name, team in teams.items():
        for pair in combinations(team, 2):
            pairs_so_far[frozenset(pair)] += 1
                
        
    return pairs_so_far

def assign_score(candidate, pairs_so_far)->int:
    """
    Description:
        Assign score (the number of past interactions) for a candidate group

    Args:
        candidate (frozenset(frozenset)): a candidate grouping

        pairs_so_far (defaultdict of frozenset: int): 
            Default dictionary where keys are frozenset of a pair and value is 
            # of past interactions, default value of 0

    Returns:
        score (int): # number of past interactions for the entire grouping
        
    """
    score = 0
    for elem in list(candidate):
        for pair in combinations(elem, 2):
            score += pairs_so_far[frozenset(pair)] 
    return score

def choose_group_splits(num_people)->dict:
    """ 
    Description:
        Choose how many groups of 4 and how many groups of 3 to generate

    Args:
        num_people (int): the number of people we're grouping

    Returns:
        split_strategy (dict[str]=int): 
            a dictionary with key group size and value number of groups
    """
    if num_people%4 == 0:
        return {"four": num_people//4, "three": 0}
    elif num_people%3 == 0:
        return {"four": 0, "three": num_people//3}
    else:
        split_strategy = {"four": 0, "three": 0}
        while num_people %3  != 0:
            num_people -= 4
            split_strategy["four"] += 1
        
        while num_people != 0:
            num_people -= 3
            split_strategy["three"] += 1
        
        return split_strategy

def generate_random_groups(people, pairs_so_far, n=1000) -> Tuple[dict, defaultdict]:
    """
    Description:
        Generate n samples of random groupings and keep track of the score. 
        This function also outputs a default dictionary that keeps track of the 
        frequency of the groupings occuring in the random sample

    Args:
        people (list[str]): a list of people for which to generate groups 

        pairs_so_far (defaultdict of frozenset: int): 
            Default dictionary where keys are frozenset of a pair and value is 
            # of past interactions, default value of 0

        n=1000 (int): # of samples to generate, default is 1000

    Returns:

        sample_score_dict (dict of frozenset(frozenset): int): 
            dictionary of key grouping and value score i.e. # of past interactions 
            among people

        repeat_samples_dict (defaultdict of frozenset(frozenset): int): 
            default dictionary of frequency of that grouping among samples generated
    """
    num_people = len(people)
    split_strategy = choose_group_splits(num_people)
    
    sample_score_dict = dict()
    repeat_samples_dict = defaultdict(int)
    for _ in range(n):
        # generate random sample by getting random order
        candidate_order = random.sample(people, num_people)
        
        g4 = candidate_order[0:split_strategy["four"]*4] 
        
        g4_2d = []
        i= 0
        while i < len(g4):
            g4_2d.append(frozenset(g4[i: i+4]))
            i+=4
        if split_strategy["three"] != 0:
            # get remaining part of list
            g3 = candidate_order[split_strategy["four"]*4:] 
            g3_2d = []
            i = 0
            while i < len(g3):
                g3_2d.append(frozenset(g3[i: i+3]))
                i+=3
        
        candidate = frozenset(g4_2d + g3_2d)
        sample_score_dict[candidate] = assign_score(candidate, pairs_so_far)
        repeat_samples_dict[candidate] += 1

    return sample_score_dict, repeat_samples_dict

def choose_best_sampled_group(sample_score_dict):
    """
    Description:
        Given the sampled groups with their scores, 
        choose a group that has the minimum score, i.e. the 
        least number of past interactions

    Args:
        sample_score_dict (dict of frozenset(frozenset): int): 
            dictionary of key grouping and value score i.e. # of past interactions 
            among people
            
    Returns:
        k (frozenset(frozenset)): 
            the best scored grouping that minimizes the number of past interactions
    """
    min_score = min(sample_score_dict.values())
    for k, v in sample_score_dict.items():
        if v == min_score:
            return k

In [28]:
def write_history(history):
    """ 
    Description:
        Write history as pickled object
    Args:
        history (dict of datetime.date: frozenset(frozenset)): the history of past groups
    Returns: None
    """
    with open('data/history.pickle', 'wb') as handle:
        pickle.dump(history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    handle.close()

def read_history():
    """
    Description:
        Read history from pickled object
    Args:
    Returns: 
        history (dict of datetime.date: frozenset(frozenset)): the history of past groups
    """
    with open('data/history.pickle', 'rb') as handle:
        history = pickle.load(handle)
    handle.close()
    return history


In [58]:
write_history(history)

In [59]:
blah = read_history(history)

In [70]:
blah[sorted(blah.keys())[-1]]

frozenset({frozenset({'Kelly', 'Mohar', 'Stan'}),
           frozenset({'Denis', 'Kirtiraj', 'Kyle B.'}),
           frozenset({'Bud', 'Eunice', 'Kyle C.'}),
           frozenset({'Carly', 'Cody', 'Piyush'}),
           frozenset({'Bridget', 'Jie', 'Jonathan'})})

In [66]:
yesterday

datetime.date(2023, 1, 27)

In [68]:
print(sorted([today, yesterday]))
print(sorted([yesterday, today]))

[datetime.date(2023, 1, 27), datetime.date(2023, 1, 28)]
[datetime.date(2023, 1, 27), datetime.date(2023, 1, 28)]


In [51]:
len(history)

1

In [26]:
blah

{'01-30-2023': frozenset({frozenset({'Carly', 'Denis', 'Eunice'}),
            frozenset({'Bud', 'Cody', 'Stan'}),
            frozenset({'Jonathan', 'Kelly', 'Kyle C.'}),
            frozenset({'Bridget', 'Kyle B.', 'Piyush'}),
            frozenset({'Jie', 'Kirtiraj', 'Mohar'})})}

In [6]:
pairs_so_far = get_pairs_so_far(history)

In [9]:
pairs_so_far[frozenset({'Kyle C.', 'Kelly'})]

1

In [11]:
scores_dict, freq = generate_random_groups(people = people, pairs_so_far = pairs_so_far, n = 1000)

In [17]:
scores_dict

{frozenset({frozenset({'Cody', 'Denis', 'Stan'}),
            frozenset({'Eunice', 'Mohar', 'Piyush'}),
            frozenset({'Bud', 'Kelly', 'Kyle B.'}),
            frozenset({'Jonathan', 'Kirtiraj', 'Kyle C.'}),
            frozenset({'Bridget', 'Carly', 'Jie'})}): 2,
 frozenset({frozenset({'Carly', 'Jie', 'Kyle C.'}),
            frozenset({'Eunice', 'Kelly', 'Kirtiraj'}),
            frozenset({'Bud', 'Cody', 'Mohar'}),
            frozenset({'Kyle B.', 'Piyush', 'Stan'}),
            frozenset({'Bridget', 'Denis', 'Jonathan'})}): 2,
 frozenset({frozenset({'Kelly', 'Kyle B.', 'Stan'}),
            frozenset({'Bud', 'Kirtiraj', 'Piyush'}),
            frozenset({'Bridget', 'Carly', 'Jie'}),
            frozenset({'Cody', 'Denis', 'Eunice'}),
            frozenset({'Jonathan', 'Kyle C.', 'Mohar'})}): 2,
 frozenset({frozenset({'Bud', 'Cody', 'Jie'}),
            frozenset({'Denis', 'Jonathan', 'Mohar'}),
            frozenset({'Kirtiraj', 'Kyle B.', 'Kyle C.'}),
            frozense

In [14]:
final_group = choose_best_sampled_group(sample_score_dict = scores_dict)

In [15]:
final_group

frozenset({frozenset({'Bridget', 'Jie', 'Kelly'}),
           frozenset({'Bud', 'Eunice', 'Piyush'}),
           frozenset({'Carly', 'Mohar', 'Stan'}),
           frozenset({'Denis', 'Kyle B.', 'Kyle C.'}),
           frozenset({'Cody', 'Jonathan', 'Kirtiraj'})})

In [71]:
newlist = [today, today + relativedelta(weekday=FR(+2))]
print(newlist)

[datetime.date(2023, 1, 28), datetime.date(2023, 2, 10)]


In [73]:
" to ".join([str(x) for x in newlist])

'2023-01-28 to 2023-02-10'